In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils import class_weight

# Set the directory paths
train_data_dir = "D:/DATASET/CNN/steatosis/train"
validation_data_dir = "D:/DATASET/CNN/steatosis/val"
test_data_dir = "D:/DATASET/CNN/steatosis/test"

# Constants
img_width, img_height = 299, 299
batch_size = 32

In [2]:
# ImageDataGenerator for training, validation, and test data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 38516 images belonging to 4 classes.
Found 38516 images belonging to 4 classes.


In [3]:
# Calculate class weights to address class imbalance
class_weights = class_weight.compute_class_weight(
    'balanced',
    np.unique(train_generator.classes),
    train_generator.classes)

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

C:\Users\JUHI\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2 3], y=[0 0 0 ... 3 3 3] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [4]:
# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(4, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze layers in base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
# Train the model with class weights
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=3,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    class_weight=dict(enumerate(class_weights)))  # Pass class weights as a dictionary

Epoch 1/3
1203/1203 [==============================] - 9620s 8s/step - loss: 1.5702 - accuracy: 0.3139 - val_loss: 0.9433 - val_accuracy: 0.5863
Epoch 2/3
1203/1203 [==============================] - 9887s 8s/step - loss: 1.3069 - accuracy: 0.4177 - val_loss: 2.0630 - val_accuracy: 0.1175
Epoch 3/3
1203/1203 [==============================] - 10371s 9s/step - loss: 1.1443 - accuracy: 0.4587 - val_loss: 0.8959 - val_accuracy: 0.6488


In [6]:
# Evaluate the model on test data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='categorical',
    shuffle=False)

Found 38516 images belonging to 4 classes.


In [12]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Importing Image module from PIL
from PIL import Image


# Predict and apply masking annotation
def predict_and_mask(image_path):
    img = Image.open(image_path)
    img = img.resize((img_width, img_height))
    img_array = np.expand_dims(np.array(img) / 255., axis=0)
    
    prediction = model.predict(img_array)
    print("Predictions: ", predictions)
    
    # Get the index of the class with maximum probability
    predicted_class = np.argmax(prediction)

    # Define the NASH score mapping
    nash_scores = {
            0: "0: < 5% steatosis area",
            1: "1: 5–33% steatosis area",
            2: "2: 33–66% steatosis area",
            3: "3: >66% steatosis area"
        }

    # Get the NASH score corresponding to the maximum probability
    nash_score = nash_scores[predicted_class]
    # Print the NASH score
    print('Predicted NASH score:', nash_score)
    
    # Apply masking
    mask = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    mask[:, :, :] = (255, 0, 0)  # Red color for masking
    
    plt.imshow(img)
    plt.imshow(mask, alpha=0.5)  # Overlay the mask with alpha blending
    plt.axis('off')
    plt.title(f'Predicted Class: {predicted_class}')
    plt.show()

ImportError: cannot import name '_OneToOneFeatureMixin' from 'sklearn.base' (C:\Users\JUHI\anaconda3\lib\site-packages\sklearn\base.py)

In [8]:
# Example usage
image_path = "D:/DATASET/CNN/steatosis/train/2/8_200_92.png"
predict_and_mask(image_path)

NameError: name 'Image' is not defined